In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
pd.options.display.max_rows = 150

First I will load the loan data, which can be downloaded as a csv from https://www.lendingclub.com/info/download-data.action

The data used represents loans that were made in years 2012 and 2013.  I only want to look at completed loans, and since there are 36 and 60 month loans, I need to go back at least 60 months to obtian completed 60 month loans.

The dependent variable is `total_pymnt`, which represents the total principal and interest paid back on the loan that will be predicted.  The prediction will be used when the loan is originated, so only the variables that would be known at that that time should be used.

In [3]:
loans = pd.read_csv('loanStats3b.csv', low_memory = False, skiprows = 1)

In [4]:
loans.shape

(188183, 145)

In [5]:
loans.iloc[:,:5].tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv
188178,NaN,NaN,35000.0,35000.0,35000.0
188179,NaN,NaN,12000.0,12000.0,12000.0
188180,NaN,NaN,12000.0,7775.0,7775.0
188181,Total amount funded in policy code 1: 2700702175,NaN,NaN,NaN,NaN
188182,Total amount funded in policy code 2: 81866225,NaN,NaN,NaN,NaN


The last 2 rows should not have been read into the dataframe and will be removed.

In [6]:
loans = loans[:-2]

Only the variables that are known at before the loan is funded will be useful for predicting the amount that will be paid, so any column that contains information that is collected after the loan is funded will be removed, except for `total_pymnt`, which is the dependent variable.

In [7]:
cols_to_remove = [
'collection_recovery_fee',
'funded_amnt_inv',
'last_pymnt_amnt',
'last_pymnt_d',
'next_pymnt_d',
'out_prncp',
'desc',
'emp_title',
'out_prncp_inv',
'pymnt_plan',
'recoveries',
'total_pymnt_inv',
'loan_amnt',
'total_rec_int',
'total_rec_late_fee',
'total_rec_prncp',
'hardship_flag',
'hardship_type',
'hardship_reason',
'hardship_status',
'deferral_term',
'hardship_amount',
'hardship_start_date',
'hardship_end_date',
'payment_plan_start_date',
'hardship_length',
'hardship_dpd',
'hardship_loan_status',
'orig_projected_additional_accrued_interest',
'hardship_payoff_balance_amount',
'hardship_last_payment_amount',
'debt_settlement_flag',
'debt_settlement_flag_date',
'settlement_status',
'settlement_date',
'settlement_amount',
'settlement_percentage',
'settlement_term']

In [8]:
loans.drop(cols_to_remove, inplace = True, axis = 1)

### Data Dictionary

In [9]:
data_dict = pd.read_excel('LCDataDictionary.xlsx', sheet_name = 'LoanStats')

In [10]:
data_dict.columns = ['Variable', 'Description']

In [11]:
variables = pd.DataFrame(loans.columns)
variables.columns = ['Variable']

In [12]:
new_dict = pd.merge(variables, data_dict)

In [13]:
print(new_dict.to_string(index=False))

Variable                                        Description
                            id      A unique LC assigned ID for the loan listing.
                     member_id   A unique LC assigned Id for the borrower member.
                   funded_amnt  The total amount committed to that loan at tha...
                          term  The number of payments on the loan. Values are...
                      int_rate                          Interest Rate on the loan
                   installment  The monthly payment owed by the borrower if th...
                         grade                             LC assigned loan grade
                     sub_grade                          LC assigned loan subgrade
                    emp_length  Employment length in years. Possible values ar...
                home_ownership  The home ownership status provided by the borr...
                    annual_inc  The self-reported annual income provided by th...
           verification_status  Indica

### Data Cleanup

I only want loans that are completed, otherwise the dependent variable (total amount paid on the loan) is unknown. I will look at the loan status to determine which loans to remove.

In [14]:
loans['loan_status'].value_counts()

Fully Paid            154671
Charged Off            29496
Current                 3807
Late (31-120 days)        97
In Grace Period           65
Late (16-30 days)         40
Default                    5
Name: loan_status, dtype: int64

Only loans with "Fully Paid" or "Charged Off" status are completed, and the rest will be removed.

In [15]:
loans = loans.query('loan_status == "Fully Paid" | loan_status == "Charged Off"')
loans.shape

(184167, 107)

Next, I will take a look at the missing value counts.

In [16]:
nulls = loans.isnull().sum()
nulls[nulls > 0]

id                                     184167
member_id                              184167
emp_length                               7715
url                                    184167
title                                       7
mths_since_last_delinq                 105379
mths_since_last_record                 167026
revol_util                                125
last_credit_pull_d                         10
mths_since_last_major_derog            152390
annual_inc_joint                       184167
dti_joint                              184167
verification_status_joint              184167
tot_coll_amt                            27741
tot_cur_bal                             27741
open_acc_6m                            184167
open_act_il                            184167
open_il_12m                            184167
open_il_24m                            184167
mths_since_rcnt_il                     184167
total_bal_il                           184167
il_util                           

There are several columns where all 184,167 observations are null. I will initially drop all columns where every value is null.

In [17]:
loans = loans.dropna(axis = 'columns', how = 'all')

In [18]:
loans.shape

(184167, 76)

The variables `title`, `revol_util` and `last_credit_pull_d` all have 150 or fewer missing values, and since the number is so small I will just drop the rows missing these values.

In [19]:
loans = loans.dropna(axis = 'rows', subset = ['title', 'revol_util', 'last_credit_pull_d'])

In [20]:
loans.shape

(184025, 76)

Next, I want to remove any columns that have only one unique value. These will not add anything to the model, however, the unique values might be useful information for other reasons, so I will create a dictionary with the discarded column names and the unique value.

In [21]:
one_unique = {}  
for col in loans.columns:
    if len(loans[col].unique()) == 1:
        one_unique.update({col: loans[col][1]})
        loans.drop(col, inplace=True, axis=1)

In [22]:
loans.shape

(184025, 73)

In [23]:
one_unique

{'policy_code': 1.0,
 'application_type': 'Individual',
 'disbursement_method': 'Cash'}

In [24]:
loans.home_ownership.value_counts()

MORTGAGE    94467
RENT        74398
OWN         15072
OTHER          46
NONE           42
Name: home_ownership, dtype: int64

In [25]:
loans.shape

(184025, 73)

There are very few observations of 'Other' and 'None', which will be dropped.

In [26]:
loans = loans.query('home_ownership == "MORTGAGE" | home_ownership == "RENT" | home_ownership == "OWN"')

In [27]:
state_val_counts = loans.addr_state.value_counts()
state_val_counts[state_val_counts < 500]

DE    460
WY    442
SD    398
VT    296
MS      3
NE      3
ID      2
IA      1
Name: addr_state, dtype: int64

In [28]:
replace_map = {'addr_state': {'DE' : 'Other', 'WY' : 'Other', 'SD' : 'Other', 'VT' : 'Other', 'NE' : 'Other', 'MS' : 'Other','ID' : 'Other', 'IA' : 'Other'}}

In [29]:
loans.replace(replace_map, inplace = True)

In [30]:
loans.shape

(183937, 73)

Next I will check the data types to see if any were read incorrectly.

In [31]:
loans.dtypes

funded_amnt                       float64
term                               object
int_rate                           object
installment                       float64
grade                              object
sub_grade                          object
emp_length                         object
home_ownership                     object
annual_inc                        float64
verification_status                object
issue_d                            object
loan_status                        object
purpose                            object
title                              object
zip_code                           object
addr_state                         object
dti                               float64
delinq_2yrs                       float64
earliest_cr_line                   object
inq_last_6mths                    float64
mths_since_last_delinq            float64
mths_since_last_record            float64
open_acc                          float64
pub_rec                           

The variables `issue_d`, `earliest_cr_line`, and `last_credit_pull_d` were read as strings but are dates. They will be converted to datetime variables.

In [32]:
date_vars = ['issue_d', 'earliest_cr_line', 'last_credit_pull_d']

In [33]:
loans.loc[:, date_vars] = loans.loc[:, date_vars].apply(pd.to_datetime, errors='coerce') 

The variables `earliest_cr_line`, and `last_credit_pull_d` would make more sense converting to number of months or years of credit history or since the last credit pull.

In [34]:
loans.shape

(183937, 73)

In [35]:
loans['days_of_credit_hist'] = (loans['issue_d'] - loans['earliest_cr_line']).dt.days


In [36]:
loans.shape

(183937, 74)

In [37]:
loans['days_since_last_cr_pull'] = (loans['issue_d'] - loans['last_credit_pull_d']).dt.days

In [38]:
loans.shape

(183937, 75)

In [39]:
loans.drop(['earliest_cr_line', 'last_credit_pull_d'], axis = 1,  inplace = True)

In [40]:
loans.shape

(183937, 73)

The variables `int_rate` and `revol_util` were read as objects, but should have been float 64.

In [41]:
print(loans.int_rate.head())
print(loans.revol_util.head())

0     10.99%
1     10.99%
2      7.62%
3     11.99%
4      6.62%
Name: int_rate, dtype: object
0    61.2%
1    16.1%
2    55.7%
3      67%
4    21.6%
Name: revol_util, dtype: object


The `int_rate` and `revol_util` variables were read with the percent symbol as a string. To correct this, I will remove the % symbol from the string and convert to numeric.

In [42]:
loans.int_rate = pd.to_numeric([x.strip('%') for x in loans.int_rate])

In [43]:
loans.revol_util = pd.to_numeric([x.strip('%') for x in loans.revol_util])

The variable `emp_length` was read as a string, but may make more sense as an integer.

In [44]:
loans.emp_length.unique()

array(['10+ years', '2 years', '3 years', '4 years', '7 years', '5 years',
       '6 years', '1 year', '8 years', '9 years', '< 1 year', nan],
      dtype=object)

Since it looks like everything greater than 10 years is grouped together as "10+ years", it makes more sense to leave this as a categorical variable since there is some built-in grouping.  I will replace the missing values with the string "none".

In [45]:
loans['emp_length'] = loans['emp_length'].fillna('none')

Next I want to look at variables with a large number of unique values.  These could be an indication of messy data that needs cleaned up.

In [46]:
high_unique = {}
for col in loans.columns:
    if len(loans[col].unique()) > 50:
        if loans[col].dtype == 'O':
            high_unique.update({col : len(loans[col].unique())})
        

In [47]:
high_unique

{'title': 44243, 'zip_code': 839}

There are two variables that have more than 50 unique values. 

In [48]:
loans[['title', 'purpose']].head(10)

,title,purpose
0,Debt Consolidation,debt_consolidation
1,For The House,home_improvement
2,Debt Consolidation and Credit Transfer,debt_consolidation
3,Debt consolidation,debt_consolidation
4,UNIVERSAL CARD,debt_consolidation
5,Debt Killer,debt_consolidation
6,Pay off,debt_consolidation
7,credit card,credit_card
8,Debt consolidation,debt_consolidation
9,Debt consolidation,debt_consolidation


`title` is a text field that input by the applicant and is quite messy.  Fortunately, it has already been cleaned up in the `purpose` variables, and `title` can be dropped.

In [49]:
loans.drop('title', inplace = True, axis = 1)

Next, I will look another look at the missing value counts.

In [50]:
nulls = loans.isnull().sum()
nulls[nulls > 0]

mths_since_last_delinq            105278
mths_since_last_record            166803
mths_since_last_major_derog       152223
tot_coll_amt                       27709
tot_cur_bal                        27709
total_rev_hi_lim                   27709
acc_open_past_24mths                7482
avg_cur_bal                        27709
bc_open_to_buy                      8884
bc_util                             8955
mo_sin_old_il_acct                 33740
mo_sin_old_rev_tl_op               27709
mo_sin_rcnt_rev_tl_op              27709
mo_sin_rcnt_tl                     27709
mort_acc                            7482
mths_since_recent_bc                8706
mths_since_recent_bc_dlq          148179
mths_since_recent_inq              27271
mths_since_recent_revol_delinq    130846
num_accts_ever_120_pd              27709
num_actv_bc_tl                     27709
num_actv_rev_tl                    27709
num_bc_sats                        16032
num_bc_tl                          27709
num_il_tl       

Twenty variables have 27,709 null values.  This could indicate a time period in which these twenty variables were not collected. To test for this, I will create a list of column names which have 27,710 null values, and add the issue date variable `issue_d`.  Then I will sort by issue date, remove the rows that consist of all null values, and see what the ealiest issue date is in the resulting data.

In [51]:
null_cols = list((nulls[nulls == 27709]).index)

In [52]:
null_cols_2 = list(null_cols)
null_cols_2.append('issue_d')

In [53]:
loans_by_date = loans.sort_values(by = 'issue_d', axis = 0, ascending = True)
loans_by_date = loans_by_date[null_cols_2]

In [54]:
loans.issue_d.unique()

array(['2013-12-01T00:00:00.000000000', '2013-11-01T00:00:00.000000000',
       '2013-10-01T00:00:00.000000000', '2013-09-01T00:00:00.000000000',
       '2013-08-01T00:00:00.000000000', '2013-07-01T00:00:00.000000000',
       '2013-06-01T00:00:00.000000000', '2013-05-01T00:00:00.000000000',
       '2013-04-01T00:00:00.000000000', '2013-03-01T00:00:00.000000000',
       '2013-02-01T00:00:00.000000000', '2013-01-01T00:00:00.000000000',
       '2012-12-01T00:00:00.000000000', '2012-11-01T00:00:00.000000000',
       '2012-10-01T00:00:00.000000000', '2012-09-01T00:00:00.000000000',
       '2012-08-01T00:00:00.000000000', '2012-07-01T00:00:00.000000000',
       '2012-06-01T00:00:00.000000000', '2012-05-01T00:00:00.000000000',
       '2012-04-01T00:00:00.000000000', '2012-03-01T00:00:00.000000000',
       '2012-02-01T00:00:00.000000000', '2012-01-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [55]:
loans_by_date.shape 

(183937, 21)

In [56]:
loans_by_date_2 = loans_by_date.dropna(axis = 'rows', subset = null_cols)

In [57]:
min(loans_by_date_2.issue_d)

Timestamp('2012-08-01 00:00:00')

Removing the rows where all of the 20 columns are null results in removing all rows with an issue date prior to 08/1/2012, which means that there is no data for the first 7 out of 24 months of data for the 20 columns with 27,710 missing values. There are several opptions for dealing with this.

First, I could impute the missing values.  This would allow these variables to remain but a fairly large proportion would be imputed.

Second, I could just remove all rows of data prior to August 2012, which would discard a lot of good data but all variables would remain with a smaller proportion of imputed values.

Third, I could just drop these 20 columns that are missing a large amount of data and only leave the columns where data was collected for the entire 2-year period.  

The only way to know which method is best would be to try them all and see which provides the best results. Initially I will go with the second option, and remove the rows prior to August 2012. 

In [58]:
loans = loans[loans.issue_d >= '2012-08-01 00:00:00']

In [59]:
loans.shape

(160815, 72)

In [60]:
nulls = loans.isnull().sum()
nulls[nulls > 0]

mths_since_last_delinq             89786
mths_since_last_record            144382
mths_since_last_major_derog       129101
tot_coll_amt                        4587
tot_cur_bal                         4587
total_rev_hi_lim                    4587
avg_cur_bal                         4587
bc_open_to_buy                      1200
bc_util                             1262
mo_sin_old_il_acct                 10618
mo_sin_old_rev_tl_op                4587
mo_sin_rcnt_rev_tl_op               4587
mo_sin_rcnt_tl                      4587
mths_since_recent_bc                1042
mths_since_recent_bc_dlq          125057
mths_since_recent_inq              17628
mths_since_recent_revol_delinq    111557
num_accts_ever_120_pd               4587
num_actv_bc_tl                      4587
num_actv_rev_tl                     4587
num_bc_tl                           4587
num_il_tl                           4587
num_op_rev_tl                       4587
num_rev_accts                       4587
num_rev_tl_bal_g

The 20 columns with the same count of null values still have null values, meaning that while data was being collected in these columns starting 08/01/12, there were still some rows since 8/1/12 with missing values. I will replace the missing values with the median for the remaining.

In [61]:
for col in null_cols:
    loans[col].fillna(loans[col].median(), inplace = True)

In [62]:
nulls = loans.isnull().sum()
nulls[nulls > 0]

mths_since_last_delinq             89786
mths_since_last_record            144382
mths_since_last_major_derog       129101
bc_open_to_buy                      1200
bc_util                             1262
mo_sin_old_il_acct                 10618
mths_since_recent_bc                1042
mths_since_recent_bc_dlq          125057
mths_since_recent_inq              17628
mths_since_recent_revol_delinq    111557
num_tl_120dpd_2m                    4839
pct_tl_nvr_dlq                      4730
percent_bc_gt_75                    1202
dtype: int64

There are a few variables that represent months since a delinquency occured, such as months since there was a delinquancy on revolving credit. In these cases, I suspect that missing values represent loans where the applicant hasn't had the delinquency, which is meaningful but is something that can't be represented numerically like the number of months.  Therefore, I think the best approach is to make this a categorical variable.


In [63]:
loans['mths_since_last_delinq'].head(12)

0      NaN
1      NaN
2      NaN
3      NaN
4     43.0
5      NaN
6     34.0
7      NaN
8     53.0
9     16.0
10    64.0
11    11.0
Name: mths_since_last_delinq, dtype: float64

In [64]:
cut_points = [0, 24, 48, 72, 1000]

In [65]:
labels = ["1-2 years", "2-4 years", "4-6 years", "Over 6 years"]

In [66]:
loans['mths_since_last_delinq'] = pd.cut(loans['mths_since_last_delinq'], bins = cut_points, labels = labels)

In [67]:
loans['mths_since_last_delinq'] = loans['mths_since_last_delinq'].cat.add_categories(['none'])
loans['mths_since_last_delinq'] = loans['mths_since_last_delinq'].fillna('none')

In [68]:
loans['mths_since_last_delinq'].head(12)

0          none
1          none
2          none
3          none
4     2-4 years
5          none
6     2-4 years
7          none
8     4-6 years
9     1-2 years
10    4-6 years
11    1-2 years
Name: mths_since_last_delinq, dtype: category
Categories (5, object): [1-2 years < 2-4 years < 4-6 years < Over 6 years < none]

In [69]:
loans['mths_since_last_delinq'].value_counts()

none            89906
1-2 years       27553
2-4 years       24864
4-6 years       13721
Over 6 years     4771
Name: mths_since_last_delinq, dtype: int64

The binning worked where there is values, but the missing values remain.  I will convert the other delinquency variables to categorical, and then replace the missing values with the category 'none'.

In [70]:
loans['mths_since_last_record'] = pd.cut(loans['mths_since_last_record'], bins = cut_points, labels = labels)

In [71]:
loans['mths_since_last_record'] = loans['mths_since_last_record'].cat.add_categories(['none'])
loans['mths_since_last_record'] = loans['mths_since_last_record'].fillna('none')

In [72]:
loans['mths_since_last_major_derog'] = pd.cut(loans['mths_since_last_major_derog'], bins = cut_points, labels = labels)

In [73]:
loans['mths_since_last_major_derog'] = loans['mths_since_last_major_derog'].cat.add_categories(['none'])
loans['mths_since_last_major_derog'] = loans['mths_since_last_major_derog'].fillna('none')

In [74]:
loans['mths_since_recent_bc_dlq'] = pd.cut(loans['mths_since_recent_bc_dlq'], bins = cut_points, labels = labels)

In [75]:
loans['mths_since_recent_bc_dlq'] = loans['mths_since_recent_bc_dlq'].cat.add_categories(['none'])
loans['mths_since_recent_bc_dlq'] = loans['mths_since_recent_bc_dlq'].fillna('none')

In [76]:
loans['mths_since_recent_revol_delinq'] = pd.cut(loans['mths_since_recent_revol_delinq'], bins = cut_points, labels = labels)

In [77]:
loans['mths_since_recent_revol_delinq'] = loans['mths_since_recent_revol_delinq'].cat.add_categories(['none'])
loans['mths_since_recent_revol_delinq'] = loans['mths_since_recent_revol_delinq'].fillna('none')

In [78]:
loans['mths_since_last_record'].value_counts()

none            144382
Over 6 years     11877
4-6 years         2590
2-4 years         1496
1-2 years          470
Name: mths_since_last_record, dtype: int64

In [79]:
loans['mths_since_last_major_derog'].value_counts()

none            129124
2-4 years        12369
4-6 years         8871
1-2 years         7592
Over 6 years      2859
Name: mths_since_last_major_derog, dtype: int64

In [80]:
loans['mths_since_recent_bc_dlq'].value_counts()

none            125105
2-4 years        13013
1-2 years         9753
4-6 years         9604
Over 6 years      3340
Name: mths_since_recent_bc_dlq, dtype: int64

In [81]:
loans['mths_since_recent_revol_delinq'].value_counts()

none            111638
2-4 years        17659
1-2 years        17517
4-6 years        10383
Over 6 years      3618
Name: mths_since_recent_revol_delinq, dtype: int64

For the remaining variables with missing values, I will replace the missing values with the median.

In [82]:

replace_with_median = [
'bc_open_to_buy',
'bc_util',
'mo_sin_old_il_acct',
'mths_since_recent_bc',
'mths_since_recent_inq',
'num_tl_120dpd_2m', 
'pct_tl_nvr_dlq',
'percent_bc_gt_75']
for col in replace_with_median:
    loans[col].fillna(loans[col].median(), inplace = True)

In [83]:
#Check to see if there are any more missing values.
nulls = loans.isnull().sum()
nulls[nulls > 0]

Series([], dtype: int64)

There are no remaining null values.  Since the manually-binned variables are of the 'category' type, I will convert the remaining categorical variables, which are of the 'object' type, to 'category' type.



In [84]:
dtypes = loans.dtypes
object_vars = list(dtypes[dtypes == 'object'].index)

In [85]:
loans[object_vars] = loans[object_vars].astype('category')

In [86]:
loans.dtypes

funded_amnt                              float64
term                                    category
int_rate                                 float64
installment                              float64
grade                                   category
sub_grade                               category
emp_length                              category
home_ownership                          category
annual_inc                               float64
verification_status                     category
issue_d                           datetime64[ns]
loan_status                             category
purpose                                 category
zip_code                                category
addr_state                              category
dti                                      float64
delinq_2yrs                              float64
inq_last_6mths                           float64
mths_since_last_delinq                  category
mths_since_last_record                  category
open_acc            

In [87]:
loans.to_csv('loans_clean.csv', index = False)